# Task Decomposition

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [5]:
from langchain_groq import ChatGroq
from langchain_ollama import ChatOllama

llm = ChatGroq(model="llama3-8b-8192")
ollama_llm = ChatOllama(model="llama3.2:1b")

In [8]:
from langchain_core.prompts import  PromptTemplate

def run_prompt(prompt, **kwargs):
    """Helper function to run a prompt through the language model.

    Args:
        prompt (str): The prompt template string.
        **kwargs: Keyword arguments to fill the prompt template.

    Returns:
        str: The model's response.
    """
    prompt_template = PromptTemplate(template=prompt, input_variables=list(kwargs.keys()))
    chain = prompt_template | llm
    return chain.invoke(kwargs).content

#Breaking down task into subtasks

In [9]:
complex_task = """
Analyze the financial health of a company based on the following data:
- Revenue: $10 million
- Net Income: $2 million
- Total Assets: $15 million
- Total Liabilities: $7 million
- Cash Flow from Operations: $3 million
"""

decomposition_prompt = """
Break down the task of analyzing a company's financial health into 3 subtasks. For each subtask, provide a brief description of what it should accomplish.

Task: {task}

Subtasks:
1.
"""
subtasks = run_prompt(decomposition_prompt, task=complex_task)
print(subtasks)

Here are the 3 subtasks to analyze a company's financial health:

**Subtask 1: Review Key Financial Ratios**

Objective: Calculate and interpret important financial ratios to understand the company's profitability, efficiency, and solvency.

This subtask should help you calculate and analyze ratios such as:

* Gross Margin Ratio (GM%)
* Operating Margin Ratio (OM%)
* Net Profit Margin Ratio (NPM%)
* Return on Equity (ROE)
* Return on Assets (ROA)
* Debt-to-Equity Ratio
* Current Ratio

By reviewing these ratios, you'll gain insights into the company's ability to generate profits, manage its assets and liabilities, and maintain financial stability.

**Subtask 2: Analyze Cash Flow and Liquidity**

Objective: Evaluate the company's ability to generate cash, manage its liquidity, and pay its debts.

This subtask should help you analyze the company's:

* Cash Flow from Operations (CFO) to understand its ability to generate cash from its core business activities.
* Current Ratio and Quick Ra

#Chaining SubTasks

In [10]:
def analyze_profitability(revenue, net_income):
    """Analyze the company's profitability.

    Args:
        revenue (float): Company's revenue.
        net_income (float): Company's net income.

    Returns:
        str: Analysis of the company's profitability.
    """
    prompt = """
    Analyze the company's profitability based on the following data:
    - Revenue: ${revenue} million
    - Net Income: ${net_income} million

    Calculate the profit margin and provide a brief analysis of the company's profitability.
    """
    return run_prompt(prompt, revenue=revenue, net_income=net_income)

In [11]:
def analyze_liquidity(total_assets, total_liabilities):
    """Analyze the company's liquidity.

    Args:
        total_assets (float): Company's total assets.
        total_liabilities (float): Company's total liabilities.

    Returns:
        str: Analysis of the company's liquidity.
    """
    prompt = """
    Analyze the company's liquidity based on the following data:
    - Total Assets: ${total_assets} million
    - Total Liabilities: ${total_liabilities} million

    Calculate the current ratio and provide a brief analysis of the company's liquidity.
    """
    return run_prompt(prompt, total_assets=total_assets, total_liabilities=total_liabilities)

In [12]:
def analyze_cash_flow(cash_flow):
    """Analyze the company's cash flow.

    Args:
        cash_flow (float): Company's cash flow from operations.

    Returns:
        str: Analysis of the company's cash flow.
    """
    prompt = """
    Analyze the company's cash flow based on the following data:
    - Cash Flow from Operations: ${cash_flow} million

    Provide a brief analysis of the company's cash flow health.
    """
    return run_prompt(prompt, cash_flow=cash_flow)

In [14]:
profitability_analysis = analyze_profitability(10, 2)
liquidity_analysis = analyze_liquidity(15, 7)
cash_flow_analysis = analyze_cash_flow(3)

print("Profitability Analysis:\n", profitability_analysis)

Profitability Analysis:
 Let's analyze the company's profitability based on the given data.

**Profit Margin:**

Profit Margin = (Net Income / Revenue) × 100

Profit Margin = ($2 million / $10 million) × 100 = 20%

**Analysis:**

The company's profit margin is 20%, which indicates that for every dollar of revenue generated, the company earns 20 cents in net income. This is a relatively healthy profit margin, indicating that the company is able to convert a significant portion of its revenue into net income.

The company's net income of $2 million represents 20% of its revenue, which suggests that it has a solid earnings performance. This could be attributed to various factors such as efficient cost management, effective pricing strategies, and a strong competitive position in the market.

However, it's essential to note that a 20% profit margin may not be exceptional for all industries. Some companies in highly competitive or low-margin industries may struggle to achieve a similar prof

In [15]:
print("\nLiquidity Analysis:\n", liquidity_analysis)


Liquidity Analysis:
 A simple but important analysis!

To calculate the current ratio, we need to divide the company's current assets by its current liabilities. However, since we don't have the current assets and current liabilities data, we'll make an assumption that the current assets are a significant portion of the total assets and current liabilities are a significant portion of the total liabilities.

Let's assume that the current assets are approximately 80% of the total assets and current liabilities are approximately 80% of the total liabilities. This is a common assumption, as current assets and current liabilities typically make up a significant portion of a company's balance sheet.

Using this assumption, we can estimate the current assets as:

Current Assets = 0.8 x $15 million = $12 million

And the current liabilities as:

Current Liabilities = 0.8 x $7 million = $5.6 million

Now, we can calculate the current ratio:

Current Ratio = Current Assets / Current Liabilitie

In [16]:
print("\nCash Flow Analysis:\n", cash_flow_analysis)


Cash Flow Analysis:
 Based on the provided data, the company's cash flow from operations is $3 million. This indicates that the company is generating a significant amount of cash from its core business activities.

Here's a brief analysis of the company's cash flow health:

* The company is generating a positive cash flow from operations, which suggests that it is able to convert its earnings into cash at a rate that is greater than or equal to its expenses.
* The $3 million in cash flow from operations is a significant amount, indicating that the company has a strong ability to generate cash from its business activities.
* This cash flow can be used to fund various aspects of the business, such as investing in new projects, paying off debt, or distributing dividends to shareholders.
* The company's ability to generate cash from operations also suggests that it has a stable and predictable revenue stream, which can help to reduce financial risk and provide a sense of stability.

Overa

#Integrating Results

In [17]:
def integrate_results(profitability, liquidity, cash_flow):
    """Integrate the results from subtasks to provide an overall analysis.

    Args:
        profitability (str): Profitability analysis.
        liquidity (str): Liquidity analysis.
        cash_flow (str): Cash flow analysis.

    Returns:
        str: Overall analysis of the company's financial health.
    """
    prompt = """
    Based on the following analyses, provide an overall assessment of the company's financial health:

    Profitability Analysis:
    {profitability}

    Liquidity Analysis:
    {liquidity}

    Cash Flow Analysis:
    {cash_flow}

    Summarize the key points and give an overall evaluation of the company's financial position.
    """
    return run_prompt(prompt, profitability=profitability, liquidity=liquidity, cash_flow=cash_flow)

overall_analysis = integrate_results(profitability_analysis, liquidity_analysis, cash_flow_analysis)
print("Overall Financial Health Analysis:\n", overall_analysis)

Overall Financial Health Analysis:
 Based on the analyses, here is a summary of the key points and an overall evaluation of the company's financial position:

**Profitability:**

* The company has a decent profit margin of 20%, indicating a good balance between revenue generation and cost management.
* The profit margin is relatively healthy, suggesting that the company is able to convert a significant portion of its revenue into net income.

**Liquidity:**

* The company has a strong liquidity position, with a current ratio of 2.14, indicating that it can easily pay its short-term debts and obligations.
* The company's ability to cover its current liabilities approximately 2.14 times suggests that it has a stable and predictable financial position.

**Cash Flow:**

* The company is generating a significant amount of cash from its core business activities, with a cash flow from operations of $3 million.
* The company's ability to generate cash from operations suggests that it has a sta